In [1]:
import sys
from nplinker import NPLinker
from logconfig import LogConfig
from metabolomics import Spectrum

In [2]:
# configuring NPLinker in a notebook env is now done either by passing in the name of a config file,
# or by passing in a dict which corresponds to the structure of the config file. Usually it will be
# easier to edit the file and simply pass the filename like this:
npl = NPLinker('latest_api_demo.toml')

# the above step will attempt to discover the files to be loaded from the dataset and complain
# if they're not as expected. Next, actually load the data files
if not npl.load_data():
    raise Exception('Failed to load data')
 

09:48:16 [WARNING] loader.py:99, mibig_json_dir "/mnt/archive/nplinker_data/carnegie_new/mibig_json" does not exist or is not readable!
Loaded 3107 molecules
Found 0 MiBIG json files
09:48:18 [WARNING] loader.py:130, Missing annotation/cluster tsv files in folder: /mnt/archive/nplinker_data/carnegie_new/bigscape/Saccharides
# MiBIG BGCs = 2383, non-MiBIG BGCS = 222, total bgcs = 2605, GCFs = 1655


In [3]:
# The scoring methods are defined and configured in the default configuration file at 
# ~/.config/nplinker/nplinker.toml, but will be overridden by the config file you loaded above,
# and the scoring methods can be easily changed once the NPLinker object has been created, e.g.:

# ensure only metcalf scoring is enabled, and set a 99% significance percentile threshold
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))
npl.scoring.likescore.enabled = False
# npl.scoring.likescore.cutoff = <scoring cutoff threshold>
npl.scoring.hg.enabled = False
# npl.scoring.hg.prob = <probability threshold>
npl.scoring.metcalf.enabled = True
npl.scoring.metcalf.sig_percentile = 99
print('Currently enabled scoring methods: {}'.format(npl.scoring.enabled()))

Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99)]
Currently enabled scoring methods: [namespace(enabled=True, name='metcalf', sig_percentile=99)]


In [4]:
# this step generates scores for all objects and enabled scoring methods, so it can be
# quite lengthy. The random_count parameter determines the number of randomised instances
# of Spectrum <=> Strain mappings that will be generated during the process.
if not npl.process_dataset(random_count=10):
    raise Exception('Failed to process dataset')
print('Completed generating scores')

Completed generating scores


In [6]:
# to get results once the scores are generated, first select an object you're interested 
# in, then call get_links with a specific scoring method. You can also pass a list of 
# objects as the first parameter. The method returns a list which contains only those
# objects that satisfy the scoring criteria (so here only those with a significance 
# percentile score of >= 99 as set above)
test_gcf = npl.gcfs[8]
results = npl.get_links(test_gcf, npl.scoring.metcalf)
if test_gcf not in results:
    print('No results found!')
else:
    print('Found results for {}!'.format(test_gcf))
    # to get the objects that scored highly against this GCF, use links_for_obj. By
    # default it will return all objects, the type_ parameter can be used to filter
    # by class, so here it will only return spectra
    test_gcf_links = npl.links_for_obj(test_gcf, npl.scoring.metcalf, type_=Spectrum)
    
    # print the objects and their scores, plus common strains
    for obj, score in test_gcf_links:
        print('{} : score {}'.format(obj, score))
        # returns a dict indexed by (Spectrum, GCF) tuples, with 
        # the values being lists of strain names shared between the two
        common_strains = npl.get_common_strains(test_gcf, obj)
        if len(common_strains) > 0:
            strain_names = list(common_strains.values())[0]
            print('   {} shared strains: {}'.format(len(strain_names), strain_names))
        else:
            print('   (no shared strains)')
            
    print('{} total links found'.format(len(test_gcf_links)))
        
    

09:49:14 [WARNING] nplinker.py:433, FIXME: MolecularFamily support
Found results for GCF(id=8, class=NRPS, gcf_id=285)!
Spectrum(id=2, spectrum_id=127) : score 26.0
   (no shared strains)
Spectrum(id=3, spectrum_id=135) : score 26.0
   (no shared strains)
Spectrum(id=6, spectrum_id=284) : score 26.0
   (no shared strains)
Spectrum(id=8, spectrum_id=328) : score 26.0
   (no shared strains)
Spectrum(id=10, spectrum_id=386) : score 26.0
   (no shared strains)
Spectrum(id=15, spectrum_id=546) : score 26.0
   (no shared strains)
Spectrum(id=43, spectrum_id=5572) : score 26.0
   (no shared strains)
Spectrum(id=50, spectrum_id=7693) : score 26.0
   (no shared strains)
Spectrum(id=55, spectrum_id=7701) : score 26.0
   (no shared strains)
Spectrum(id=56, spectrum_id=7703) : score 26.0
   (no shared strains)
Spectrum(id=59, spectrum_id=7706) : score 26.0
   (no shared strains)
Spectrum(id=61, spectrum_id=7709) : score 26.0
   (no shared strains)
Spectrum(id=72, spectrum_id=7723) : score 26.0
   